In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import time

# First Web Scrapping

Usamos classes para todos os métodos de Web Scrapping pois compactam bem o código.

In [5]:
class Scrapper:
    def __init__(self, url_pattern, content_parser=None):
        self.url_pattern = url_pattern
        self.content_parser = content_parser

    def scrape_url(self, url):
        response = requests.get(url, timeout=5)
        if response.status_code < 300:
            result = self.content_parser(response.content)[1:]
            self.output_results(result)
        else: print(f"fail. error code: {response.status_code}")

    #para cada categoria do site criamos um arquivo diferente, inclusive para a categoria geral que reúne todas
    def output_results(self, r):
        if CATEGORY == '': file = open(f'omelete-noticias-geral.csv','w+')
        else: file = open(f'omelete-noticias-{CATEGORY[9:]}.csv','w+')
        for item in r:
            file.write(f'{item}\n')
        file.close()

    def kickstart(self):
        self.scrape_url(self.url_pattern)


#observamos que poderíamos reduzir a área de busca na sopa para evitar retornar lixo
def parser(content):
    soup = list(bs(content).children)[14]
    quoteshtml = soup.find_all('h2')
    return [quoteshtml[k].get_text() for k in range(len(quoteshtml))]

#utilizamos uma url base que era comum a todas as cinco categorias que buscamos
BASE_URL = 'https://www.omelete.com.br/criticas'
CATEGORY_LIST = ['', '?section=filmes', '?section=series-tv', '?section=quadrinhos', '?section=musica']
URL_PATTERN = ''

#utilizamos um looping para rodar por todas as categorias
for CATEGORY in CATEGORY_LIST:
    URL_PATTERN = f'{BASE_URL}{CATEGORY}'
    spider = Scrapper(URL_PATTERN, content_parser=parser)
    spider.kickstart()

IndexError: list index out of range

# Second Web Scrapping

In [11]:
class Scrapper:
    def __init__(self, url_pattern, content_parser=None):
        self.url_pattern = url_pattern
        self.content_parser = content_parser

    def scrape_url(self, url):
        response = requests.get(url, timeout=5)
        if response.status_code < 300:
            result = self.content_parser(response.content)
            self.output_results(result)
        else: print(f"fail. error code: {response.status_code}")

    def output_results(self, r):
        file = open('top100-movies-rt.csv', 'w+')
        file.write(f'ranking,rating,movie_name,release_year,total_reviews')
        file.write('\n')
        k = 0
        for minilist in r:
            k += 1
            file.write(f'{k},{minilist[0]},{minilist[1]},{minilist[2]},{minilist[3]}\n')
        file.close()

    def kickstart(self):
        self.scrape_url(self.url_pattern)

#reduzimos a área de busca na sopa mas ainda assim percebemos que a lista carregava lixo
#por este motivo, limitamos o retorno aos elementos de índice 1 ao -20
#desta forma retornamos exatamente a lista de 100 filmes
#também procuramos deixar todas as informações numéricas, incluindo porcentagens, em variáveis apenas numéricas
def parser(content):
    soup = list(list(bs(content).children)[2].children)[3]
    infos = soup.find_all('tr', class_='')[1:-20]
    #cortamos os 13 espaços em branco no começo do título do filme os últimos 7 caracteres referentes ao ano
    #colocamos o ano em uma coluna separada
    return [[infos[k].find_all('span', class_='tMeterScore')[0].get_text()[1:].strip('%'),
            infos[k].find_all('a', class_='unstyled articleLink')[0].get_text()[13:-7],
            infos[k].find_all('a', class_='unstyled articleLink')[0].get_text().split(' ')[-1].strip('()'),
            infos[k].find_all('td', class_='right hidden-xs')[0].get_text()] for k in range(len(infos))]

BASE_URL = 'https://www.rottentomatoes.com/top/bestofrt/'

spider = Scrapper(BASE_URL, content_parser=parser)
spider.kickstart()

AttributeError: 'NavigableString' object has no attribute 'children'

# Third Web Scrapping

In [7]:
class Scrapper:
    def __init__(self, url_pattern, content_parser=None):
        self.url_pattern = url_pattern
        self.content_parser = content_parser

    def scrape_url(self, url):
        response = requests.get(url, timeout=5)
        if response.status_code < 300:
            result = self.content_parser(response.content)
            self.output_results(result)
        else: print(f"fail. error code: {response.status_code}")

    #como aqui precisávamos limpar as informações mais a fundo preferimos criar uma lista antes de imprimir
    #também adicionamos a categoria na lista para termos o nome da personagem nos dados
    def output_results(self, r):
        infos = r
        infos.append(CATEGORY)
        classes_info.append(infos)

    def kickstart(self):
        self.scrape_url(self.url_pattern)

#reduzimos a área de busca da sopa para evitar retorno de lixo
#percebemos que para três categorias específicas havia um valor a mais e precisamos fazer um retorno diferente
def parser(content):
    soup = list(list(list(bs(content).children)[2].children)[3].children)[26]
    stats = soup.find_all('td')
    if CATEGORY not in ['Thief', 'Dragoon', 'Mime']: return [item.get_text().strip(' ').strip('\n') for item in stats[5:13]]
    else: return [item.get_text().strip(' ').strip('\n') for item in stats[6:14]]

classes_info = []
#utilizamos uma url base que era comum a todas as categorias que iríamos buscar
BASE_URL = 'https://finalfantasy.fandom.com/wiki/'
CATEGORY_LIST = ['Squire', 'Chemist', 'Knight', 'Archer', 'White_Mage',
                 'Black_Mage', 'Monk', 'Thief', 'Mystic', 'Time_Mage',
                 'Geomancer', 'Dragoon', 'Orator', 'Summoner', 'Samurai',
                 'Ninja', 'Arithmetician', 'Dancer', 'Bard', 'Mime',
                 'Dark_Knight', 'Onion_Knight', 'Holy_Knight', 'Machinist', 'Skyseer',
                 'Netherseer', 'Divine_Knight', 'Sword_Saint', 'Templar', 'Dragonkin',
                 'Soldier', 'Sky_Pirate', 'Game_Hunter']

#utilizamos um looping para rodar todas as categorias que queríamos
#mas percebemos que especificamente para 5 categorias o link não seguia o padrão exato dos outros e
#precisamos fazer um retorno diferente
#por precaução, como eram mais de 30 acessos, colocamos um intervalo de 1 segundo para evitar quaisquer problemas
for CATEGORY in CATEGORY_LIST:
    if CATEGORY not in ['Arithmetician', 'Skyseer', 'Netherseer', 'Dragonkin', 'Game_Hunter']: URL_PATTERN = f'{BASE_URL}{CATEGORY}_(Tactics)'
    else: URL_PATTERN = f'{BASE_URL}{CATEGORY}'
    spider = Scrapper(URL_PATTERN, content_parser=parser)
    spider.kickstart()
    time.sleep(1)

AttributeError: 'NavigableString' object has no attribute 'children'

In [8]:
#uma das linhas continha uma categoria com informações duplas pois, dependendo da condição que estava aplicada,
#o valor seria diferente. assim, criamos uma linha nula logo após e preenchemos ela com as informações extras
#ao mesmo tempo que as removíamos da linha anterior
classes_info.insert(22, [0,0,0,0,0,0,0,0,0])
for k in range(len(classes_info[21])):
    item = classes_info[21][k]
    try:
        classes_info[21][k] = item.split(',')[0].strip(' (Base)')
        classes_info[22][k] = item.split(',')[1].strip(' (Mastered)')
    except:
        if item == 'Onion_Knight':
            classes_info[21][k] = item
            classes_info[22][k] = f'{item}_Mastered'
        else:
            classes_info[21][k] = item
            classes_info[22][k] = item

IndexError: list index out of range

In [9]:
#também classificamos a taxa de crescimento de atributos por um índice numérico indo de 1 a 5
#no lugar de categorias que iam de "Very Low" a "Very High"
#removemos o símbolo de porcentagem dos dados de uma coluna para ela ser totalmente numérica
for job in classes_info:
    for stat in job:
        if stat == 'Very High': job[job.index(stat)] = 5
        elif stat == 'High': job[job.index(stat)] = 4
        elif stat in ['Medium', 'Average']: job[job.index(stat)] = 3
        elif stat == 'Low': job[job.index(stat)] = 2
        elif stat == 'Very Low': job[job.index(stat)] = 1
        if '%' in stat: job[job.index(stat)] = int(stat.strip('%'))

TypeError: argument of type 'int' is not iterable

In [10]:
#por fim imprimimos o arquivo
file = open(f'final-fantasy-tactics-classes-info.csv','w+')
file.write(f'class_name,move_rate,jump_rate,speed,evasion_(%),base_attack,base_magic,base_hitpoints,base_manapoints\n')
for item in classes_info:
    file.write(f'{item[8].lower()},{item[0]},{item[1]},{item[2]},{item[3]},{item[4]},{item[5]},{item[6]},{item[7]}\n')
file.close()

AttributeError: 'int' object has no attribute 'lower'

# First API

In [ ]:
#acessamos um api que retorna informações do jogo civilization e pedimos as informações de civilizações
response = requests.get('https://age-of-empires-2-api.herokuapp.com/api/v1/civilizations')
response

In [ ]:
#filtramos a parte do json que nos traria as informações de cada civilização
civilizations = response.json()
civilizations['civilizations']

In [ ]:
#imprimimos as informações que julgamos relevantes
file = open('age2.csv', 'w+') 
file.write('civiliation_name,army_type,unique_unit,unique_tech,team_bonus\n')
for item in civilizations['civilizations']:
    file.write(f"{item['name']},{item['army_type']},{item['unique_unit'][0].split('/')[-1]},{item['unique_tech'][0].split('/')[-1]},{item['team_bonus']}\n")
file.close()    

# Second API

In [ ]:
#acessamos um api que retorna informações de asteróides
#pedimos as informações de 100 asteróides com órbita circular ou o mais próximo disso
response = requests.get('http://asterank.com/api/asterank?query={"e":{"$lt":0.1}}&limit=100')
response

In [ ]:
asteroids = response.json()
asteroids

In [ ]:
#imprimimos todas as informações desses asteróides. precisamos de um looping para imprimir as chaves como
#nomes de coluna e de outro looping para imprimir as informações de cada asteróide
file = open('asteroids.csv', 'w+')
for title in list(dic.keys()):
    file.write(f'{title}')
    if title != 'n_dop_obs_used': file.write(',')
file.write('\n')
for dic in asterois:
    for k in range(len(list(dic.values()))):
        file.write(f'{list(dic.values())[k]}')
        if k < 81: file.write(',')
    file.write('\n')
file.close()

# Third API

## First Part

In [ ]:
#acessamos um api que retorna informações sobre futebol e pedimos as informações de competições
response = requests.get('http://api.football-data.org/v2/competitions/')
response

In [ ]:
#filtramos a parte do json que nos traria as informações de cada competição
competitions = response.json()
competitions['competitions']

In [ ]:
#imprimimos os arquivos. precisamos especificar os casos em que o campeonato ainda não tinha nenhum vencedor
#e também o caso em que a região buscada não estava com nenhum campeonato disponível
file = open(f'football-data-competitions.csv','w+')
file.write(f"id,area,competition_name,season_start_date,season_end_date,winner\n")
for dic in competitions['competitions']:
    try:
        if dic['currentSeason']['winner'] == None: file.write(f"{dic['id']},{dic['area']['name']},{dic['name']},{dic['currentSeason']['startDate']},{dic['currentSeason']['endDate']},{dic['currentSeason']['winner']}\n")
        else: file.write(f"{dic['id']},{dic['area']['name']},{dic['name']},{dic['currentSeason']['startDate']},{dic['currentSeason']['endDate']},{dic['currentSeason']['winner']['name']}\n")
    except: pass
file.close()

## Second Part

In [ ]:
#nesta parte pedimos as informações de todas as equipes do campeonato "Série A" no brasil
#era necessário fornecer uma chave de acesso para obter as informações aqui
response = requests.get('http://api.football-data.org/v2/competitions/2013/teams', headers={'X-Auth-Token':'46f2a41057a649a59e5412649b2e279b'})
response

In [ ]:
teams = response.json()
teams

In [ ]:
#imprimimos as informações que julgamos relevantes para cada time
file = open(f'football-data-teams-brazil.csv','w+')
file.write(f"id,tla,name,founded,venue,website\n")
for dic in teams['teams']:
    file.write(f"{dic['id']},{dic['tla']},{dic['name']},{dic['founded']},{dic['venue']},{dic['website']}\n")
file.close()

## Third Part

In [ ]:
#por fim, pedimos todas as informações de todos os times do campeonato "Série A" no brasil
#e selecionamos apenas as informações de cada jogador. junto a elas colocamos para qual time cada um jogava
#para evitar que o site nos bloqueasse pois eram mais de 800 acessos contínuos colocamos um intervalo
#de 5 segundos entre cada pedido (tentamos com valores menores que ainda nos prendiam)
teams_info = []

for team_id in teams['teams']:
    response = response.get(f"http://api.football-data.org/v2/teams/{team_id['id']}", headers={'X-Auth-Token':'46f2a41057a649a59e5412649b2e279b'})
    team_info = response.json()
    for dic in team_info['squad']:
        teams_info.append([team_id['id'], dic['id'], dic['name'], dic['position'], dic['nationality']])
    time.sleep(5)

In [ ]:
#por fim, imprimimos tudo
file = open(f'football-data-teams-brazil-squad.csv','w+')
file.write(f'team_id,player_id,player_name,player_position,player_country\n')
for item in teams_info:
    file.write(f'{item[0]},{item[1]},{item[2]},{item[3]},{item[4]}\n')
file.close()